In [1]:
datFile =  "shape_predictor_68_face_landmarks.dat"

In [2]:
import cv2 
import dlib

cap = cv2.VideoCapture(0)

hog_face_detector = dlib.get_frontal_face_detector()

dlib_facelandmark = dlib.shape_predictor(datFile)

while True:
    frame = cap.read() 
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = hog_face_detector(gray)
    for face in faces:

        face_landmarks = dlib_facelandmark(gray, face)

        for n in range(0, 16):
            x = face_landmarks.part(n).x
            y = face_landmarks.part(n).y
            cv2.circle(frame, (x, y), 1, (0, 255, 255), 1)


    cv2.imshow("Face Landmarks", frame)

    key = cv2.waitKey(1)
    if key == 27:
        break
    cap.release() 
    cv2.destroyAllWindows()

error: OpenCV(4.6.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numerical tuple
>  - Expected Ptr<cv::UMat> for argument 'src'


In [3]:
import cv2
import numpy as np
import dlib
from math import hypot
from heatmap import heatmap

cap = cv2.VideoCapture(0)

detector = dlib.get_frontal_face_detector()

predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

font = cv2.FONT_HERSHEY_PLAIN

def midpoint(p1, p2):
    return int((p1.x + p2.x) / 2), int((p1.y + p2.y) / 2)

def get_eye(eye_points, facial_landmarks):
    left_point = (facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y)
    right_point = (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y)

    center_top = midpoint(facial_landmarks.part(eye_points[1]), facial_landmarks.part(eye_points[2]))
    center_bottom = midpoint(facial_landmarks.part(eye_points[5]), facial_landmarks.part(eye_points[4]))

    hor_line_length = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))
    ver_line_length = hypot((center_top[0] - center_bottom[0]), (center_top[1] - center_bottom[1]))

    eye_x = int((left_point[0] + right_point[0]) / 2)
    eye_y = int((center_top[1] + center_bottom[1]) / 2)

    cv2.circle(frame, (eye_x, eye_y), 5, (0, 0, 255), 2)

    ratio = hor_line_length / ver_line_length

    return ratio, (eye_x, eye_y)

def get_gaze_ratio(eye_points, facial_landmarks, gray_eye):
    eye_region = np.array([
        (facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y),
        (facial_landmarks.part(eye_points[1]).x, facial_landmarks.part(eye_points[1]).y),
        (facial_landmarks.part(eye_points[2]).x, facial_landmarks.part(eye_points[2]).y),
        (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y),
        (facial_landmarks.part(eye_points[4]).x, facial_landmarks.part(eye_points[4]).y),
        (facial_landmarks.part(eye_points[5]).x, facial_landmarks.part(eye_points[5]).y)], np.int32)

    h, w = gray_eye.shape

    mask = np.zeros((h, w), np.uint8)

    cv2.polylines(mask, [eye_region], True, 255, 2)
    cv2.fillPoly(mask, [eye_region], 255)
    eye = cv2.bitwise_and(gray_eye, gray_eye, mask=mask)

    _, threshold_eye = cv2.threshold(eye, 40, 255, cv2.THRESH_BINARY)
    h, w = threshold_eye.shape

    left_side_threshold = threshold_eye[0:h, 0:int(w / 2)]
    left_side_white = cv2.countNonZero(left_side_threshold)

    right_side_threshold = threshold_eye[0:h, int(w / 2):w]
    right_side_white = cv2.countNonZero(right_side_threshold)

    if left_side_white == 0:
        gaze_ratio = 1
    elif right_side_white == 0:
        gaze_ratio = 5
    else:
        gaze_ratio = left_side_white / right_side_white

    return gaze_ratio

def get_screen_coordinates(gaze_point, screen_width, screen_height):
    gaze_x, gaze_y = gaze_point
    screen_x = int(gaze_x * screen_width)
    screen_y = int(gaze_y * screen_height)
    return screen_x, screen_y

screen_width, screen_height = 1920, 1080  # Adjust with your screen resolution

heatmap_data = np.zeros((screen_height, screen_width), np.uint8)

while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)

    for face in faces:
        landmarks = predictor(gray, face)

        left_eye_ratio, left_eye_point = get_eye(list(range(36, 42)), landmarks)
        right_eye_ratio, right_eye_point = get_eye(list(range(42, 48)), landmarks)

        left_gaze_ratio = get_gaze_ratio(list(range(36, 42)), landmarks, gray)
        right_gaze_ratio = get_gaze_ratio(list(range(42, 48)), landmarks, gray)

        if left_gaze_ratio <= 1.5 and right_gaze_ratio <= 1.5:
            cv2.putText(frame, "Center", (50, 100), font, 2, (0, 0, 255), 3)
        elif left_gaze_ratio > 1.5:
            screen_x, screen_y = get_screen_coordinates(left_eye_point, screen_width, screen_height)
            cv2.putText(frame, f"Left ({screen_x}, {screen_y})", (50, 100), font, 2, (0, 0, 255), 3)
            heatmap_data[screen_y, screen_x] += 1
        elif right_gaze_ratio > 1.5:
            screen_x, screen_y = get_screen_coordinates(right_eye_point, screen_width, screen_height)
            cv2.putText(frame, f"Right ({screen_x}, {screen_y})", (50, 100), font, 2, (0, 0, 255), 3)
            heatmap_data[screen_y, screen_x] += 1

    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()

heatmap(heatmap_data, output_file="heatmap.png")


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
